In [294]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import  time
import numpy as np

In [295]:
def get_html(url):
    res = requests.get(url,
                       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'})
    res.encoding = 'utf-8'
    return res

In [296]:
def get_one_page(url):
    res =get_html(url)
    soup = bs(res.text , 'html.parser')
    divs = soup.find_all(class_='hd')
    hrefs = [each.a['href'] for each in divs]
    return hrefs

In [297]:
href_data = []
for i in range(10):
    url = f'https://movie.douban.com/top250?start={i*25}&filter='
    hrefs = get_one_page(url)
    href_data+=hrefs

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
pd.Series(href_data).to_csv('hrefs.csv', index=False ,header=False)

In [ ]:
def get_detail_page(url):
    data = []
    res = get_html(url)
    soup = bs(res.content,'html.parser')
    
    try:
        content_div = soup.find('div', id = 'content')
    except:
        content_div = ''
    
    try:
        name = content_div.find('span',property = 'v:itemreviewed').string.split(' ')[0]
    except:
        name = ''
    
    try:
        director = content_div.find('a',rel='v:directedBy').string
    except:
        director = ''
    
    try:
        scriptwriter = content_div.find_all('span',class_='attrs')[1].text.replace(' ','')
    except:
        scriptwriter = ''
    
    try:
        actors = content_div.find_all('span',class_='attrs')[2].text.replace(' ','')
    except:
        actors = ''
    
    try:
        kind_span = soup.find_all('span',property='v:genre')
        kind_tmp = ''
        for i in range(len(kind_span)):
            kind_tmp += kind_span[i].text+'/'
        kind = kind_tmp[:-1]
    except:
        kind = ''
    
    try:
        pn_country = re.compile(r'<span class="pl">制片国家/地区:</span> (.*?)<br/>',re.S)
        country = re.findall(pn_country,str(content_div))[0].replace(' ','')
    except:
        country = ''
    
    try:
        pn_languege = re.compile(r'<span class="pl">语言:</span> (.*?)<br/>',re.S)
        languege = re.findall(pn_languege,str(content_div))[0]
    except:
        languege = ''
    
    try:
        releasedate_span = soup.find_all('span',property='v:initialReleaseDate')
        releasedate_tmp = ''
        for i in range(len(releasedate_span)):
            releasedate_tmp += releasedate_span[i].text+'/'
        releasedate = releasedate_tmp[:-1]
    except:
        releasedate = ''
    
    try:
        runtime = soup.find('span',property='v:runtime').string
    except:
        runtime = ''
    
    try:
        rating_num = soup.find('strong',class_='ll rating_num').string
    except:
        rating_num = ''
    
    try:
        rating_people = soup.find('span',property='v:votes').string
    except:
        rating_people = ''
    
    try:
        tag_div = soup.find('div',class_='tags-body')
        tag_a = tag_div.find_all('a')
        tag_tmp = ''
        for i in range(len(tag_a)):
            tag_tmp += tag_a[i].text+'/'
        tags = tag_tmp[:-1]
    except:
        tags = ''
    
    data = [name,director,scriptwriter,actors,kind,country,languege,releasedate,runtime,rating_num,rating_people,tags]
    return data

In [ ]:
# get_detail_page('https://movie.douban.com/subject/1292052/')

In [ ]:
def main(start,end):
    df = pd.DataFrame(columns=['电影名称','导演','编剧','演员','类型','国家','语言','上映日期','时长','评分','评论人数','标签'])
    for index in range(start,end+1):
        time.sleep(np.random.uniform(30,60))
        print(f'正在抓取第{index}部电影的数据')
        url = href_data[index-1]
        data = get_detail_page(url)
        df.loc[index-1] = data
        
    df.to_csv('豆瓣top250.csv',mode='a')
    print('抓取结束')
        

In [ ]:
main(1,250)